### Import Required Packages and Set Options

In [83]:
import os
import sys

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
from itertools import combinations
from functools import partial

import jax.numpy as jnp
from jax import grad, jit, vmap

from pandas.api.types import CategoricalDtype
from scipy.sparse import csr_matrix

from sklearn.base import BaseEstimator, TransformerMixin

In [2]:
REPO_ROOT = "/Users/ericlundquist/Repos/rankfm"
DATA_ROOT = os.path.join(REPO_ROOT, "data/ml-100k")

### Load Example Data

In [3]:
users_df = pd.read_csv(os.path.join(DATA_ROOT, "users.csv"))
items_df = pd.read_csv(os.path.join(DATA_ROOT, "items.csv"))
ratings_df = pd.read_csv(os.path.join(DATA_ROOT, "ratings.csv"))

print("users: {} items: {} ratings: {}".format(users_df.shape, items_df.shape, ratings_df.shape))

users: (943, 5) items: (1682, 21) ratings: (100000, 4)


#### Prepare Users Data

In [4]:
users_df['agegroup'] = pd.cut(users_df['age'], [0, 30, 45, 100], right=False, labels=False)
users_df = users_df.drop(['age', 'zip_code'], axis=1)
users_df = pd.get_dummies(users_df, prefix_sep='__', columns=['agegroup', 'gender', 'occupation'])
users_df.mean()

user_id                      472.000000
agegroup__0                    0.433722
agegroup__1                    0.348887
agegroup__2                    0.217391
gender__F                      0.289502
gender__M                      0.710498
occupation__administrator      0.083775
occupation__artist             0.029692
occupation__doctor             0.007423
occupation__educator           0.100742
occupation__engineer           0.071050
occupation__entertainment      0.019088
occupation__executive          0.033934
occupation__healthcare         0.016967
occupation__homemaker          0.007423
occupation__lawyer             0.012725
occupation__librarian          0.054083
occupation__marketing          0.027572
occupation__none               0.009544
occupation__other              0.111347
occupation__programmer         0.069989
occupation__retired            0.014846
occupation__salesman           0.012725
occupation__scientist          0.032874
occupation__student            0.207847


#### Prepare Items Data

In [5]:
item_names = items_df[['item_id', 'item_name']]
item_names.head()

,item_id,item_name
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [6]:
items_df = items_df.drop(['item_name', 'release_date'], axis=1)
items_df.columns = ['item_id'] + ["genre__{}".format(col) for col in items_df.columns[1:]]
items_df.mean()

item_id               841.500000
genre__action           0.149227
genre__adventure        0.080262
genre__animation        0.024970
genre__childrens        0.072533
genre__comedy           0.300238
genre__crime            0.064804
genre__documentary      0.029727
genre__drama            0.431034
genre__fantasy          0.013080
genre__film_noir        0.014269
genre__horror           0.054697
genre__musical          0.033294
genre__mystery          0.036266
genre__romance          0.146849
genre__scifi            0.060048
genre__thriller         0.149227
genre__war              0.042212
genre__western          0.016052
dtype: float64

#### Prepare Ratings Data

In [7]:
ratings_df['timestamp'] = pd.to_datetime(ratings_df['unix_timestamp'], origin='unix', unit='s')
ratings_df = ratings_df.drop('unix_timestamp', axis=1)
ratings_df.mean()

user_id    462.48475
item_id    425.53013
rating       3.52986
dtype: float64

### Build the Sparse Model Matrix

In [102]:
class RankFM():
    """Factorization Machines for Ranking Problems with Implicit Feedback Data"""
    
    def __init__(self, factors=10, regularization=0.1, learning_rate=0.1, sigma=0.1):
        """store input hyperparameters and initialize internal data elements"""
        
        # hyperparameters
        self.factors = factors
        self.regularization = regularization
        self.learning_rate = learning_rate
        self.sigma = sigma
        
        # key column names
        self.user_id_ = None
        self.item_id_ = None
        self.rating_ = None
        
        # unique feature names
        self.users_ = None
        self.items_ = None
        self.user_features_ = None
        self.item_featuers_ = None
        
        # required internal data structures
        self.column_index_ = None
        self.model_matrix_ = None
        self.rating_vector_ = None
        
        # model weights
        self.w_constant_ = None
        self.w_features_ = None
        self.w_factors_ = None
        
        
    def initialize_weights(self):
        """initialize model weights after the model matrix has been prepared"""
        
        w_user = np.zeros(len(self.users_))
        w_item = np.zeros(len(self.items_))
        w_user_feature = np.zeros(len(self.user_features_))
        w_item_feature = np.zeros(len(self.item_features_))

        self.w_constant_ = self.rating_vector_.mean()
        self.w_features_ = np.concatenate([w_user, w_item, w_user_feature, w_item_feature])
        self.w_factors_  = np.random.normal(loc=0, scale=self.sigma, size=(len(self.w_features_), self.factors))
        
        
    def prepare_data(self, interactions, user_features, item_features):
        """build the column index, model matrix, and rating vector from components

        :param interactions: dataframe of [user_id, item_id, rating] records 
        :param user_features: dataframe of [user_id, user_feature_1, ... , user_feature_n] records
        :param item_features: dataframe of [item_id, item_feature_1, ... , item_feature_n] records
        :return: self
        """

        # identify the [user_id, item_id, rating] column names
        self.user_id_, self.item_id_, self.rating_ = interactions.columns

        # store the interactions data and assign a row index to use for the model matrix
        interactions_ = interactions.assign(row_index=np.arange(len(interactions)))
        self.rating_vector_ = interactions_[self.rating_].values

        # store unique feature names by namespace 
        self.users_ = np.sort(interactions_[self.user_id_].unique())
        self.items_ = np.sort(interactions_[self.item_id_].unique())
        self.user_features_ = np.sort(user_features.columns[1:])
        self.item_features_ = np.sort(item_features.columns[1:])

        # create namespace-specific column name lists
        user_cols = pd.DataFrame({'namespace': 'users', 'col_name': self.users_})
        item_cols = pd.DataFrame({'namespace': 'items', 'col_name': self.items_})
        user_features_cols = pd.DataFrame({'namespace': 'user_features', 'col_name': self.user_features_})
        item_features_cols = pd.DataFrame({'namespace': 'item_features', 'col_name': self.item_features_})

        # combine the column names from all namespaces and map to model matrix column indexes
        self.column_index_ = pd.concat([user_cols, item_cols, user_features_cols, item_features_cols], axis=0, ignore_index=True)
        self.column_index_['col_index'] = np.arange(len(self.column_index_))

        # separate the column names / indexes by namespace
        user_columns = self.column_index_[self.column_index_['namespace'] == 'users']
        item_columns = self.column_index_[self.column_index_['namespace'] == 'items']
        user_feature_columns = self.column_index_[self.column_index_['namespace'] == 'user_features']
        item_feature_columns = self.column_index_[self.column_index_['namespace'] == 'item_features']

        # reshape (user_features, item_features) long and drop zero-valued features to save space  
        user_features_long = pd.melt(user_features, id_vars=self.user_id_, var_name='feature_name', value_name='value').query('value > 0')
        item_features_long = pd.melt(item_features, id_vars=self.item_id_, var_name='feature_name', value_name='value').query('value > 0')

        # join the (user_features, item_features) with the interactions matrix to get dataframes at the [user_id, item_id, feature_id] level
        user_features_long = pd.merge(interactions_, user_features_long, on=self.user_id_, how='inner')
        item_features_long = pd.merge(interactions_, item_features_long, on=self.item_id_, how='inner')

        # create the component (row, col, val) tuples necessary to build the final sparse model matrix
        user_tuples = pd.merge(interactions_, user_columns, left_on=self.user_id_, right_on='col_name', how='inner')[['row_index', 'col_index']].assign(value=1)
        item_tuples = pd.merge(interactions_, item_columns, left_on=self.item_id_, right_on='col_name', how='inner')[['row_index', 'col_index']].assign(value=1)
        user_features_tuples = pd.merge(user_features_long, user_feature_columns, left_on='feature_name', right_on='col_name', how='inner')[['row_index', 'col_index', 'value']]
        item_features_tuples = pd.merge(item_features_long, item_feature_columns, left_on='feature_name', right_on='col_name', how='inner')[['row_index', 'col_index', 'value']]
        all_tuples = pd.concat([user_tuples, item_tuples, user_features_tuples, item_features_tuples], axis=0, ignore_index=True)

        # create the final sparse model matrix
        mm_input = (all_tuples['value'], (all_tuples['row_index'], all_tuples['col_index']))
        mm_shape = (len(interactions_), len(self.column_index_))
        self.model_matrix_ = csr_matrix(mm_input, shape=mm_shape)

        # calculate model matrix metadata
        interaction_sparsity = round(1 - (self.model_matrix_.shape[0] / (len(self.users_) * len(self.items_))), 4)
        nonzero_entries = self.model_matrix_.count_nonzero()
        storage_size = round(self.model_matrix_.data.nbytes / 1e6, 2)
        print(interaction_sparsity, nonzero_entries, storage_size)

        # initialize model weights and return object reference
        self.initialize_weights()
        return self
    
    
    def predict(self, X):
        """generate predicted ratings given input data rows and current model weights"""

        # create a generator object enumerating the index positions of all second-order interactions
        index_pairs = combinations(self.column_index_.col_index.values, 2)

        # calculate the predicted rating as the sum of three terms
        term_1 = self.w_constant_
        term_2 = jnp.dot(X, self.w_features_).T
        term_3 = jnp.hstack([jnp.dot(self.w_factors_[i], self.w_factors_[j]) * jnp.multiply(X[:,i], X[:,j]) for i, j in index_pairs if X[:,i].sum() and X[:,j].sum()]).sum(axis=1)

        predictions = term_1 + term_2 + term_3
        return predictions
    

    def loss_value(self, w_constant, w_features, w_factors, X, y):
        """calculate regularized squared error loss"""
    
        predictions = self.predict(X)
        mse = jnp.mean(jnp.square(y - predictions))
        penalty = sum([jnp.sum(self.regularization * jnp.square(w)) for w in [w_constant, w_features, w_factors]])

        loss = mse + penalty
        return loss

    
    # auto-diff the loss function wrt all weights
    loss_gradient = grad(loss_value, argnums=[1, 2, 3])
            
    
    def fit(self, X, y, batch_size=10):
        """update model weights by mini-batch"""
        
        # create zipped (X, y) batch tuples for training
        n_batches = np.round(X.shape[0] / batch_size)
        batch_x = np.array_split(X, n_batches)
        batch_y = np.array_split(y, n_batches)
        batches = zip(batch_x, batch_y)
        
        # update model weights one batch at a time
        for batch_x, batch_y in batches:
            gradients = self.loss_gradient(self.w_constant_, self.w_features_, self.w_factors_, batch_x, batch_y)
            self.w_constant_ -= self.learning_rate * gradients[0]
            self.w_features_ -= self.learning_rate * gradients[1]
            self.w_factors_  -= self.learning_rate * gradients[2]
            
        # return object reference with updated model weights
        return self
        

#### Initialize the Model Object

In [103]:
rankfm = RankFM()
rankfm

#### Build the Model Matrix and Initialize the Weights

In [104]:
interactions_df = ratings_df[['user_id', 'item_id', 'rating']]
rankfm.prepare_data(interactions_df, users_df, items_df)

0.937 712585 5.7


#### Perform a Batch Update

In [105]:
n_train = 20

In [106]:
X = rankfm.model_matrix_[:n_train].todense()
y = rankfm.rating_vector_[:n_train]

In [107]:
loss = rankfm.loss_value(rankfm.w_constant_, rankfm.w_features_, rankfm.w_factors_, X, y)
print("loss before batch update: {}".format(loss))

loss before batch update: 30.074390411376953


In [108]:
rankfm.fit(X, y, batch_size=10)

In [109]:
loss = rankfm.loss_value(rankfm.w_constant_, rankfm.w_features_, rankfm.w_factors_, X, y)
print("loss after batch update: {}".format(loss))

loss after batch update: 27.746797561645508


### It's Alive!

# START SANDBOX CODE

#### Do Some Diagnostic Checks

In [700]:
column_index.head()

,namespace,col_name,col_index
0,const,const,0
1,users,1,1
2,users,2,2
3,users,3,3
4,users,4,4


In [62]:
column_index.col_index.values

array([   0,    1,    2, ..., 2666, 2667, 2668])

In [701]:
column_index.namespace.value_counts()

items            1682
users             943
user_features      26
item_features      18
const               1
Name: namespace, dtype: int64

In [702]:
type(model_matrix), model_matrix.shape

(scipy.sparse.csr.csr_matrix, (100000, 2670))

In [703]:
1 + users.shape[0] + items.shape[0] + (users.shape[1] - 1) + (items.shape[1] - 1)

2670

In [704]:
interactions.head()

,user_id,item_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


In [706]:
rating_vector.head()

,rating
row_index,
0,3
1,3
2,1
3,2
4,1


In [710]:
row_number = 3
nonzero_columns = mm_sample.iloc[row_number, :].T.reset_index()
nonzero_columns[nonzero_columns[row_number] > 0]

,col_name,3
0,const,1
244,244,1
994,51,1
2626,agegroup__0,1
2630,gender__M,1
2650,occupation__technician,1
2659,genre__drama,1
2665,genre__romance,1
2668,genre__war,1
2669,genre__western,1


In [711]:
users[users.user_id == 244]

,user_id,agegroup__0,agegroup__1,agegroup__2,gender__F,gender__M,occupation__administrator,occupation__artist,occupation__doctor,occupation__educator,...,occupation__marketing,occupation__none,occupation__other,occupation__programmer,occupation__retired,occupation__salesman,occupation__scientist,occupation__student,occupation__technician,occupation__writer
243,244,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [712]:
items[items.item_id == 51]

,item_id,genre__action,genre__adventure,genre__animation,genre__childrens,genre__comedy,genre__crime,genre__documentary,genre__drama,genre__fantasy,genre__film_noir,genre__horror,genre__musical,genre__mystery,genre__romance,genre__scifi,genre__thriller,genre__war,genre__western
50,51,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1
